In [1]:
import pandas as pd
import numpy as np

In [26]:
seqlen1 = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/AllseqsUpto150k.txt',sep ='\t')
seqlen2 = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/AllseqsUnderto150k_withRef.txt',sep ='\t')
seqlen =pd.concat([seqlen1,seqlen2])

In [27]:
correia = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/data/EBV/descriptions/TableS2-Correia2017-AccNumb.csv',skiprows=4)

# drop columns completely nulls
ind = ~correia.apply(lambda x: all(x.isnull()),axis = 0)
correia = correia.loc[:,ind]
correia = correia.loc[:,['Sample Name','Geographic origin','Sample type','Type 2','cluster 2']]
correia.rename(columns ={'cluster 2':'AccId'},inplace = True)
correia = pd.merge(correia,seqlen,how = 'left',on = 'AccId')


In [28]:
chaira2016 = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/data/EBV/descriptions/TableS3Chaira2016.csv',skiprows=3)
chaira2016.columns
chaira2016 = chaira2016.loc[~chaira2016.apply(lambda x: all(x.isnull()),axis = 1),:]
chaira2016 = chaira2016.loc[:,(chaira2016.columns !='Unnamed: 8')] # drop this column that contains very heterogeneous and confusing data
chaira2016 = chaira2016.loc[:,['accession number','name as displayed in figure 1A',
       'Geographic origin','reference publication','type','ebna2 type',
       'lmp1 type','group']]
#chaira2016[chaira2016['reference publication']=='this study']['type'][]
chaira2016.head()

,accession number,name as displayed in figure 1A,Geographic origin,reference publication,type,ebna2 type,lmp1 type,group
1,LN827551,LN827551_Ken_1,Kenya,Palser et al 2015,BL,1.1,Med,1.0
2,LN827550,LN827550_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
3,LN827552,LN827552_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
4,LN827558,LN827558_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
5,LN827565,LN827565_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0


In [29]:
both = pd.merge(correia,chaira2016,left_on='AccId',right_on='accession number',how = 'inner',suffixes=['_corr','_chai'])
both = both.fillna(0)
both.head()

,Sample Name,Geographic origin_corr,Sample type,Type 2,AccId,seqlen,accession number,name as displayed in figure 1A,Geographic origin_chai,reference publication,type,ebna2 type,lmp1 type,group
0,Saliva1,UK,Healthy saliva (wild type),0.0,LN824142,171730.0,LN824142,LN824142_UK_1,UK,Palser et al 2015,Healthy saliva,1.1,Chi,8.0
1,HL01,UK,Hodgkin's lymphoma,0.0,LN824226,171797.0,LN824226,LN824226_UK_1,UK,Palser et al 2015,HL,1.1,Chi,3.0
2,HL02,UK,Hodgkin's lymphoma,0.0,LN827546,171896.0,LN827546,LN827546_UK_1,UK,Palser et al 2015,HL,1.3,Chi,5.0
3,HL04,UK,Hodgkin's lymphoma,0.0,LN827564,171943.0,LN827564,LN827564_UK_1,UK,Palser et al 2015,HL,1.1,Chi,7.0
4,HL05,UK,Hodgkin's lymphoma,0.0,LN824204,171774.0,LN824204,LN824204_UK_1,UK,Palser et al 2015,HL,1.1,Chi,7.0


In [30]:
aux = both[['Sample type','type']]
sample_mapp ={}
for i in range(len(aux['Sample type'])):
    sample_mapp.update({aux['Sample type'][i]:aux['type'][i]})
#sample_mapp 


In [33]:
corrFull = correia[correia.seqlen >100000]

In [34]:
corrFull['Sample type'].replace(sample_mapp).value_counts(dropna = False)

sLCL                                   47
BL tumour biopsy                       14
BL                                     13
Gastric carcinoma                       9
NPC biopsy                              9
HL                                      7
sLCL,IM                                 5
Lung carcinoma                          4
NPC                                     2
NPC - saliva                            1
sLCL. PTLD fatal                        1
BL cell line                            1
GC cell line SNU-719                    1
LCL (from B95-8, deletion in EBER2)     1
IM                                      1
HL cell line                            1
NPC - tumour biopsy                     1
NPC cell line                           1
GC cell line                            1
NPC -tumour                             1
Healthy saliva                          1
LCL,NPC virus                           1
NaN                                     1
Name: Sample type, dtype: int64

In [38]:
corrFull['Type 2'].value_counts(dropna = False)

NaN    124
Name: Type 2, dtype: int64

In [66]:
#aux = corrFull[
typ = corrFull['Sample type'].str
aux = corrFull[typ.contains('sLCL') | typ.contains('BL') | typ.contains('NPC')]
aux.groupby(['Geographic origin', 'Sample type'])['AccId'].count().unstack().fillna(0)

Sample type,BL,BL cell line,BL tumour biopsy,LCL from NPC virus,NPC,NPC - saliva,NPC - tumour biopsy,NPC -tumour,NPC biopsy,NPC cell line,sLCL,sLCL. IM,sLCL. IM.,sLCL. PTLD,sLCL. PTLD early,sLCL. PTLD fatal
Geographic origin,,,,,,,,,,,,,,,,
Africa,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"Africa, Nigeria",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Argentina,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,16.0,0.0,0.0
Brazil,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
China,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
Ghana,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Japan,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0, 1, 2, 3, 4, 6, 7, 8]